In [57]:
!pip install --upgrade setuptools
!pip install librosa noisereduce hmmlearn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/124.7 kB ? eta -:--:--
   --- ------------------------------------ 10.2/124.7 kB ? eta -:--:--
   --------- ----------------------------- 30.7/124.7 kB 445.2 kB/s eta 0:00:01
   ------------ -------------------------- 41.0/124.7 kB 281.8 kB/s eta 0:00:01
   ---------------------------- ---------- 92.2/124.7 kB 479.1 kB/s eta 0:00:01
   ---------------------------------- --- 112.6/124.7 kB 469.7 kB/s eta 0:00:01
   -------------------------------------- 124.7/124.7 kB 431.8 kB/s eta 0:00:00


In [69]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import noisereduce as nr
import soundfile as sf
import os
import json
from sklearn.model_selection import train_test_split
from hmmlearn import hmm
from sklearn.preprocessing import StandardScaler

preprocesing

In [29]:
def preprocess_audio(file_path, output_path, target_sampling_rate=16000):
    # Load the audio file
    audio, sampling_rate = librosa.load(file_path, sr=target_sampling_rate)

    # Noise reduction
    reduced_noise_audio = nr.reduce_noise(y=audio, sr=sampling_rate)

    # Silence removal
    non_silent_audio, _ = librosa.effects.trim(reduced_noise_audio, top_db=20) # top_db: threshold for silence in dB

    # Normalize the audio to a standard volume level
    normalized_audio = librosa.util.normalize(non_silent_audio)

    # Save the processed audio file
    sf.write(output_path, normalized_audio, target_sampling_rate)


In [32]:
input_directory = 'recordings/'
preprocessed_directory = 'preprocessed_recordings/'

for filename in os.listdir(input_directory):
    if filename.endswith('.wav'):  # Assuming your files are in wav format
        file_path = os.path.join(input_directory, filename)
        output_path = os.path.join(preprocessed_directory, filename)
        
        preprocess_audio(file_path, output_path)

Feature extraction

In [97]:
# Function to extract MFCCs from an audio file
def extract_mfcc(file_path, n_mfcc=13, n_fft=2048, hop_length=512):
    audio, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    return mfccs


# Function to plot a heatmap for MFCCs
def plot_mfcc_heatmap(mfccs, title):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mfccs, x_axis='time', cmap='viridis')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.tight_layout()
    plt.show()

In [1]:
# Directory containing preprocessed recordings
directory = 'preprocessed_recordings'
# Iterate over files in the directory and process each file
for filename in os.listdir(directory):
    if filename.endswith(".wav"):
        # Construct full file path
        file_path = os.path.join(directory, filename)
        
        # Extract digit, speaker name, and index from the filename
        parts = filename.split('_')
        digit_label, speaker_name, index = parts[0], parts[1], parts[2].split('.')[0]
        
        # Extract MFCCs
        mfccs = normalize_features(extract_mfcc(file_path))
        
        # Plot heatmap for the MFCCs
        title = f"MFCCs for {digit_label} by {speaker_name} (Index {index})"
        plot_mfcc_heatmap(mfccs, title)

NameError: name 'os' is not defined

In [91]:
import os
from sklearn.model_selection import train_test_split

def ensure_minimum_representation(files, test_size=0.2):
    if len(files) == 1:
        # If only one file, it goes into training; consider duplicating or using augmentation if needed for testing
        return files, files
    elif len(files) == 2:
        # If two files, one goes into each set
        return [files[0]], [files[1]]
    else:
        # More than two files, ensure at least one in each set and split the rest
        train, test = train_test_split(files, test_size=test_size, random_state=42)
        return train, test

input_dir = 'preprocessed_recordings'
speaker_digit_files = {}

# Organize files by speaker and digit
for filename in os.listdir(input_dir):
    if filename.endswith(".wav"):
        digit, speaker, _ = filename.rsplit('_', 2)
        speaker_digit_key = f"{speaker}_{digit}"
        if speaker_digit_key not in speaker_digit_files:
            speaker_digit_files[speaker_digit_key] = []
        speaker_digit_files[speaker_digit_key].append(filename)

train_files = []
test_files = []

# Ensure representation in both sets
for files in speaker_digit_files.values():
    train_subset, test_subset = ensure_minimum_representation(files)
    train_files.extend(train_subset)
    test_files.extend(test_subset)

# Here train_files and test_files have at least one representation of each speaker-digit pair
# You might need to adjust paths or handle the file saving/loading based on your setup

print(f"Train files count: {len(train_files)}, Test files count: {len(test_files)}")


Train files count: 2400, Test files count: 600


HMMLEARN

In [86]:
input_dir = 'preprocessed_recordings'
mfcc_dir = 'mfcc_normalized'  # Directory to save normalized and length-adjusted MFCCs
if not os.path.exists(mfcc_dir):
    os.makedirs(mfcc_dir)

speaker_digit_files = {}

# This part remains the same as your original code
# Organizing files by speaker and digit
for filename in os.listdir(input_dir):
    if filename.endswith(".wav"):
        digit, speaker, _ = filename.rsplit('_', 2)
        speaker_digit_key = f"{speaker}_{digit}"
        if speaker_digit_key not in speaker_digit_files:
            speaker_digit_files[speaker_digit_key] = []
        speaker_digit_files[speaker_digit_key].append(filename)

# Modifying this part to include MFCC extraction, normalization, and saving
train_files, test_files = [], []

for speaker_digit, files in speaker_digit_files.items():
    train_subset, test_subset = ensure_minimum_representation(files)
    
    # Process train files
    for train_file in train_subset:
        mfccs = extract_mfcc(os.path.join(input_dir, train_file))
        mfccs_normalized = normalize_features(mfccs)
        np.save(os.path.join(mfcc_dir, train_file.replace('.wav', '.npy')), mfccs_normalized)
        train_files.append(train_file.replace('.wav', '.npy'))
    
    # Process test files
    for test_file in test_subset:
        mfccs = extract_mfcc(os.path.join(input_dir, test_file))
        mfccs_normalized = normalize_features(mfccs)
        np.save(os.path.join(mfcc_dir, test_file.replace('.wav', '.npy')), mfccs_normalized)
        test_files.append(test_file.replace('.wav', '.npy'))

print(f"Train files count: {len(train_files)}, Test files count: {len(test_files)}")


Train files count: 2400, Test files count: 600


In [ ]:
class HMM:
    def __init__(self, num_hidden_states):
        self.num_hidden_states = num_hidden_states
        self.rand_state = np.random.RandomState(1)

        self.initial_prob = self._normalize(self.rand_state.rand(self.num_hidden_states, 1))
        self.transition_matrix = self._stochasticize(self.rand_state.rand(self.num_hidden_states, self.num_hidden_states))

        self.mean = None
        self.covariances = None
        self.num_dimensions = None

    def _forward(self, observation_matrix):
        log_likelihood = 0.
        T = observation_matrix.shape[1]
        alpha = np.zeros(observation_matrix.shape)

        for t in range(T):
            if t == 0:
                ## TODO: Forward algorithm for the first time step
            else:
                ## TODO: Forward algorithm for the next time steps

            alpha_sum = np.sum(alpha[:, t])
            alpha[:, t] /= alpha_sum
            log_likelihood += np.log(alpha_sum)

        return log_likelihood, alpha

    def _backward(self, observation_matrix):
        T = observation_matrix.shape[1]
        beta = np.zeros(observation_matrix.shape)

        beta[:, -1] = np.ones(observation_matrix.shape[0])

        for t in range(T - 1)[::-1]:
            beta[:, t] = ## TODO: Backward algorithm for the time steps of the HMM
            beta[:, t] /= np.sum(beta[:, t])

        return beta

    def _state_likelihood(self, obs):
        obs = np.atleast_2d(obs)
        B = np.zeros((self.num_hidden_states, obs.shape[1]))

        for s in range(self.num_hidden_states):
            np.random.seed(self.rand_state.randint(1))
            B[s, :] = ## TODO: Compute the likelihood of observations with multivariate normal pdf

        return B

    def _normalize(self, x):
        return (x + (x == 0)) / np.sum(x)

    def _stochasticize(self, x):
        return (x + (x == 0)) / np.sum(x, axis=1)

    def _em_init(self, obs):
        if self.num_dimensions is None:
            self.num_dimensions = obs.shape[0]
        if self.mean is None:
            subset = self.rand_state.choice(np.arange(self.num_dimensions), size=self.num_hidden_states, replace=False)
            self.mean = obs[:, subset]
        if self.covariances is None:
            self.covariances = np.zeros((self.num_dimensions, self.num_dimensions, self.num_hidden_states))
            self.covariances += np.diag(np.diag(np.cov(obs)))[:, :, None]

        return self

    def _em_step(self, obs):
        obs = np.atleast_2d(obs)
        T = obs.shape[1]

        B = ## TODO

        log_likelihood, alpha = ## TODO
        beta = ## TODO

        xi_sum = np.zeros((self.num_hidden_states, self.num_hidden_states))
        gamma = np.zeros((self.num_hidden_states, T))

        for t in range(T - 1):
            partial_sum = ## TODO
            xi_sum += self._normalize(partial_sum)
            partial_g = ## TODO
            gamma[:, t] = self._normalize(partial_g)
        partial_g = ## TODO
        gamma[:, -1] = self._normalize(partial_g)

        expected_prior = ## TODO
        expected_transition = self._stochasticize(## TODO)

        expected_covariances = np.zeros((self.num_dimensions, self.num_dimensions, self.num_hidden_states))
        expected_covariances += .01 * np.eye(self.num_dimensions)[:, :, None]

        gamma_state_sum = np.sum(gamma, axis=1)
        gamma_state_sum = gamma_state_sum + (gamma_state_sum == 0)

        expected_mean = np.zeros((self.num_dimensions, self.num_hidden_states))
        for s in range(self.num_hidden_states):
            gamma_obs = obs * gamma[s, :]
            expected_mean[:, s] = np.sum(gamma_obs, axis=1) / gamma_state_sum[s]

        self.initial_prob = expected_prior
        self.mean = expected_mean
        self.covariances = expected_covariances
        self.transition_matrix = expected_transition

        return log_likelihood

    def train(self, obs, num_iterations=1):
        for i in range(num_iterations):
            self._em_init(obs)
            self._em_step(obs)
        return self

    def score(self, obs):
        B = self._state_likelihood(obs)
        log_likelihood, _ = self._forward(B)
        return log_likelihood


In [ ]:
 hmm_models = []

## TODO 
# Iterate through each unique genre in the training data
# extract features from the audio files, and train a HMM for
# each genre, storing the trained models in the hmm_models list.


In [ ]:
## TODO
# In this section, iterate through the audio files, extract
# features, and use the trained HMMs to predict the genre
# for each audio file